In [3]:
train_path = "/mnt/data_file/robot/Q3_images/images"
processed_path = "/mnt/data_file/robot/Q3_images/processed_data"

In [4]:
import os
import shutil
from random import choices

In [5]:
options = [True, False]
weights = [0.8, 0.2]
train_dog_path = os.path.join(processed_path, "training", "dog")
train_cat_path = os.path.join(processed_path, "training", "cat")
test_dog_path = os.path.join(processed_path, "testing", "dog")
test_cat_path = os.path.join(processed_path, "testing", "cat")
for image in os.listdir(train_path):
    train = choices(options, weights)[0]
    if not image[0].isupper():
        if train:
            shutil.copy(os.path.join(train_path, image), os.path.join(train_dog_path, image))
        else:
            shutil.copy(os.path.join(train_path, image), os.path.join(test_dog_path, image))
    else:
        if train:
            shutil.copy(os.path.join(train_path, image), os.path.join(train_cat_path, image))
        else:
            shutil.copy(os.path.join(train_path, image), os.path.join(test_cat_path, image))
        

In [6]:
def train():
    import json
    import tempfile
    from tensorflow.keras import layers
    from tensorflow.keras import models
    from tensorflow.keras import optimizers
    from tensorflow.keras.callbacks import ModelCheckpoint
    import tensorflow as tf
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    with strategy.scope():

    
        checkpoint_path = "/mnt/data_file/ass4.model/robot/"
        random_path = tempfile.TemporaryDirectory()
    
        chief_callback = ModelCheckpoint(checkpoint_path, 
                                         monitor='acc', 
                                         save_best_only=True,
                                         mode='max')
        dummy_callback = ModelCheckpoint(random_path.name, monitor='acc', save_best_only=True)
    
    
        processed_path = "/mnt/data_file/robot/Q3_images/processed_data"
        
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu',
                                input_shape=(150, 150, 3)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        
        model.compile(loss='binary_crossentropy',
                      optimizer=optimizers.RMSprop(learning_rate=1e-4),
                      metrics=['acc'])
    
        from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
        train_dir = os.path.join(processed_path, "training")
        validation_dir = os.path.join(processed_path, "testing")
        
        # All images will be rescaled by 1./255
        train_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen = ImageDataGenerator(rescale=1./255)
        train_generator = train_datagen.flow_from_directory(
                # This is the target directory
                train_dir,
                # All images will be resized to 150x150
                target_size=(150, 150),
                batch_size=64,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')
        validation_generator = test_datagen.flow_from_directory(
                validation_dir,
                target_size=(150, 150),
                batch_size=64,
                class_mode='binary')
        if 'TF_CONFIG' in os.environ:    
            tf_config = json.loads(os.environ['TF_CONFIG'])    
            node_index = tf_config['task']['index']    
            is_chief = node_index == 0    
            print(f"Node Index: {node_index}, Is Chief: {is_chief}")
        callback = [chief_callback if is_chief else dummy_callback]


        model.fit(
          train_generator,
        #cuz there are 2000 sapmples=100*20(batch-size(20images))
          #steps_per_epoch=100,
          epochs=5,
          validation_data=validation_generator,
          callbacks=callback)

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master("spark://spark:7077") \
                    .appName("Ass4-Q3-robot") \
                    .config("spark.executor.memory", "8g") \
                    .getOrCreate()

from spark_tensorflow_distributor import MirroredStrategyRunner
 
BATCH_SIZE_PER_REPLICA = 64
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
runner = MirroredStrategyRunner(num_slots=2, local_mode=False, use_gpu=False, use_custom_strategy=True)
runner.run(train)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 05:24:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/30 05:24:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Doing CPU training...
Will run with 2 Spark tasks.
Distributed training in progress...
View Spark executor stderr logs to inspect training...
23/11/30 05:24:15 WARN DAGScheduler: Barrier stage in job 0 requires 2 slots, but only 0 are available. Will retry up to 40 more times
23/11/30 05:24:30 WARN DAGScheduler: Barrier stage in job 0 requires 2 slots, but only 0 are available. Will retry up to 39 more times
23/11/30 05:24:45 WARN DAGScheduler: Barrier stage in job 0 requires 2 slots, but only 0 are available. Will retry up to 38 more times
23/11/30 05:25:00 WARN DAGScheduler: Barrier stage in job 0 requires 2 slots, but

In [8]:
import tensorflow as tf
checkpoint_path = "/mnt/data_file/ass4.model/robot/"
model = tf.keras.models.load_model(checkpoint_path)

2023-11-30 05:47:29.136654: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 05:47:29.177916: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 05:47:29.618037: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 05:47:29.620086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 05:47:31.288646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [20]:
training_dir = "/mnt/data_file/robot/Q3_images/processed_data/training"
testing_dir = "/mnt/data_file/robot/Q3_images/processed_data/testing"
validation_dir = "/mnt/data_file/kaggle/train_data/processed_data/training"

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def report(folder):
    image_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                folder,
                target_size=(150, 150),
                shuffle=False,
                batch_size=1,
                class_mode='binary')
    model.evaluate(image_generator)

In [16]:
report(training_dir)

Found 5887 images belonging to 2 classes.
5887/5887 [==============================] - 73s 12ms/step - loss: 0.5052 - acc: 0.7588


In [17]:
report(testing_dir)

Found 1503 images belonging to 2 classes.
1503/1503 [==============================] - 20s 13ms/step - loss: 0.5082 - acc: 0.7705


In [21]:
report(validation_dir)

Found 20075 images belonging to 2 classes.
20075/20075 [==============================] - 198s 10ms/step - loss: 0.6529 - acc: 0.6332
